In [ ]:
from astropy.coordinates import SkyCoord
from astroquery.mast import Observations
import os
import gzip, shutil, glob

In [ ]:
def Galex(pos, size = 0.1, unzip = True):
    """
    This function simplifies the fits file download process from GALEX.

    Parameters
    ----------
    pos : astropy.coordinates.sky_coordinate.SkyCoord
        position information generated by astropy. It could be either from name or Ra and Dec.
    size : float, optional
        The searching cone size, by default 0.1 [deg]
    unzip : bool, optional
        Whether to unzip the downloaded file, by default True.
    """

    # Full list
    obs_table = Observations.query_region(pos,radius= size)
    data_products_by_obs = Observations.get_product_list(obs_table[:])
    # query list
    data_products = data_products_by_obs[(data_products_by_obs['obs_collection'] == 'GALEX')]

    # Preventing from repeating download
    # Here we only download three datatypes: intensity map, sky background map and background subtracted image
    a = data_products.to_pandas()
    b = a[a.productFilename.str.contains('int.fits|skybg.fits|intbgsub.fits')]

    # Start downloading
    for _, row in b.iterrows():
        Observations.download_file(row.dataURI)

    if unzip:
        for file in glob.glob("*.gz"):
            with gzip.open(file, 'r') as f_in, open(file.replace(".gz", ""), 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)  # unzipped gz files
            os.remove(file)   # delete gz files  
    else:
        pass

In [ ]:
Galex(pos = SkyCoord.from_name("PTF09gn"))